### * RNN 주요 레이어 종류
#### (1) SimpleRNN :가장 간단한 형태의 RNN레이어, 활성화 함수로 tanh가 사용됨(tanh: -1 ~ 1 사이의 값을 반환)
#### (2) LSTM(Long short Term Memory) : 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제), LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함, 활성화 함수로 tanh외에 sigmoid가 사용됨
#### (3) GRU(Gated Recurent Unit) : 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수함

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Sequence data
X = np.array([[0,1,2,3],
              [1,2,3,4],
              [2,3,4,5],
              [3,4,5,6],
              [4,5,6,7],
              [5,6,7,8]],dtype=np.float32)

x_data = tf.reshape(X,(-1,4,1))  # (6,4,1)

y_data = np.array([4,5,6,7,8,9],dtype=np.float32)

print(x_data.shape,y_data.shape)
# print(type(x_data),type(y_data))
x_data

(6, 4, 1) (6,)


<tf.Tensor: shape=(6, 4, 1), dtype=float32, numpy=
array([[[0.],
        [1.],
        [2.],
        [3.]],

       [[1.],
        [2.],
        [3.],
        [4.]],

       [[2.],
        [3.],
        [4.],
        [5.]],

       [[3.],
        [4.],
        [5.],
        [6.]],

       [[4.],
        [5.],
        [6.],
        [7.]],

       [[5.],
        [6.],
        [7.],
        [8.]]], dtype=float32)>

### [1] SimpleRNN
#### 가장 간단한 형태의 RNN

In [3]:
# RNN 순환 신경만 구현 : SimpleRNN

model = tf.keras.Sequential([
    # X: (N,D) , Wx:(D,H) Wh: (H,H) b: H
    #    (6,1) ,    (1,300)     (300,300)    300    --> 1*300+300*300+300 = 90600 param
    # (N,T,D) : (6,4,1) --> (N,T,H) : (6,4,300) , T 는 sequence_lenth, H 는  Hidden Size
    # return_sequences=True는 3차원(N,T,D)으로 출력
    # return_sequences=False는 2차원(N,H)으로 출력, 기본값
    tf.keras.layers.SimpleRNN(units=300, return_sequences=True, input_shape=(4,1)), #(?,4,1) ?는 batch_size가 얼만지 모르기 때문에 입력 X
    tf.keras.layers.SimpleRNN(units=300),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',loss='mse')
model.summary()                             

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4, 300)            90600     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 300)               180300    
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 271,201
Trainable params: 271,201
Non-trainable params: 0
_________________________________________________________________


In [4]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100)
print(model.predict(x_data))

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 35.0873
Epoch 2/100
1/1 [==============================] - 0s 10ms/step - loss: 0.9879
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 4.9095
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: 6.1622
Epoch 5/100
1/1 [==============================] - 0s 9ms/step - loss: 3.3662
Epoch 6/100
1/1 [==============================] - 0s 8ms/step - loss: 1.5376
Epoch 7/100
1/1 [==============================] - 0s 9ms/step - loss: 2.0641
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 2.9874
Epoch 9/100
1/1 [==============================] - 0s 9ms/step - loss: 2.7487
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 1.8289
Epoch 11/100
1/1 [==============================] - 0s 9ms/step - loss: 1.1060
Epoch 12/100
1/1 [==============================] - 0s 9ms/step - loss: 0.8980
Epoch 13/100
1/1 [==============================] - 0s 8ms/s

In [5]:
# 학습되지 않은 입력 데이터에 대한 예측 결과
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))

1/1 [==============================] - 0s 13ms/step
[[9.49235]]
1/1 [==============================] - 0s 11ms/step
[[1.2670304]]


In [6]:
# 평가
model.evaluate(x_data, y_data)

1/1 [==============================] - 0s 142ms/step - loss: 0.0049


0.0049368334002792835

### [2] LSTM(Long short Term Memory)
#### 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제)
#### LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함

In [7]:
# RNN 순환 신경망 구현 : LSTM

model = tf.keras.Sequential([
    # X: (N,D) , Wx:(D,4H)  Wh: (4H,H)    b: 4H
    #    (6,1) ,    (1,300)     (300,300)    4*300    --> 1*4*300+4*300*300+4*300 = 4*90600 param
    # (N,T,D) : (6,4,1) --> (N,T,H) : (6,4,300) , T 는 sequence_lenth, H 는  Hidden Size
    
    tf.keras.layers.LSTM(units=300, return_sequences=True, input_shape=[4,1]), #(?,4,1) ?는 batch_size가 얼만지 모르기 때문에 입력 X
    tf.keras.layers.LSTM(units=300),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',loss='mse')
model.summary()                             

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 300)            362400    
                                                                 
 lstm_1 (LSTM)               (None, 300)               721200    
                                                                 
 dense_1 (Dense)             (None, 1)                 301       
                                                                 
Total params: 1,083,901
Trainable params: 1,083,901
Non-trainable params: 0
_________________________________________________________________


In [8]:
# 학습 및 예측
model.fit(x_data,y_data,epochs=100,verbose= 0)
print(model.predict(x_data))

1/1 [==============================] - 0s 305ms/step
[[3.3811972]
 [5.109454 ]
 [6.3834367]
 [7.3105726]
 [7.99787  ]
 [8.518563 ]]


### [3] GRU(Gated Recurent Unit)
#### 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수

In [9]:
# RNN 순환 신경망 구현  : GRU
model = tf.keras.Sequential([
    tf.keras.layers.GRU(units=300, return_sequences=True, input_shape=[4,1]),
    tf.keras.layers.GRU(units=300),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 4, 300)            272700    
                                                                 
 gru_1 (GRU)                 (None, 300)               541800    
                                                                 
 dense_2 (Dense)             (None, 1)                 301       
                                                                 
Total params: 814,801
Trainable params: 814,801
Non-trainable params: 0
_________________________________________________________________
